# XGBoost trainer

This notebook function handles training and logging of xgboost models **only**, exposing both the sklearn and low level api's.

## steps
1. generate an xgboost model configuration by selecting one of 5 available types
2. get a sample of data from a data source (random rows, consecutive rows, or the entire dataset)
3. split the data into train, validation, and test sets.  

> _PLEASE NOTE_:  there are many approaches to cross validation (cv) and as many ways to implement cv in scikit learn.  In this third stage, an alternative, two-way train and test split can be created.  The training set would then, for example, serve as input to a cross validation splitter.  The latter creates multiple training and validation subsets, called folds. These folds are then input, either in sequence or in parallel into the fit algorithm.

4. train the model
5. dump the model
6. generate predictions and probabilities
7. (calibrate probabilities if needed, wip)
8. calculate evaluation statistics and plots

All these steps have been separated here into independent functions since many can be reused for other model types. Some of the following functions will be transferred in the `mlrun.mlutils` module. Additionally, each function contains its own imports in order to isolate and identify dependencies.

In [1]:
# nuclio: ignore
import nuclio

In [2]:
%nuclio config kind = "job"
%nuclio config spec.image = "mlrun/ml-models"

%nuclio: setting kind to 'job'
%nuclio: setting spec.image to 'mlrun/ml-models'


In [3]:
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

In [4]:
from  mlrun.mlutils import (get_sample,
                            get_splits,
                            gen_sklearn_model,
                            create_class, 
                            eval_class_model)
from mlrun.execution import MLClientCtx
from mlrun.datastore import DataItem
from cloudpickle import dumps

## generate an xgb model

generate a model config using the xgboost's sklearn api

In [5]:
def gen_xgb_model(model_type: str, xgb_params: dict):
    """generate an xgboost model
    
    Multiple model types that can be estimated using
    the XGBoost Scikit-Learn API.
    
    Input can either be a predefined json model configuration or one
    of the five xgboost model types: "classifier", "regressor", "ranker",
    "rf_classifier", or "rf_regressor".
    
    In either case one can pass in a params dict to modify defaults values.
    
    Based on `mlrun.mlutils.models.gen_sklearn_model`, see the function
    `sklearn_classifier` in this repository.
    
    :param model_type: one of "classifier", "regressor",
                       "ranker", "rf_classifier", or
                      "rf_regressor"
    :param xgb_params: class init parameters
    """
    from mlrun.mlutils import get_class_fit, gen_sklearn_model

    # generate model and fit function
    mtypes = {
        "classifier"   : "xgboost.XGBClassifier",
        "regressor"    : "xgboost.XGBRegressor",
        "ranker"       : "xgboost.XGBRanker",
        "rf_classifier": "xgboost.XGBRFClassifier",
        "rf_regressor" : "xgboost.XGBRFRegressor"
    }
    if model_type.endswith("json"):
        model_config = model_type
    elif model_type in mtypes.keys():
        model_config = mtypes[model_type]
    else:
        raise Exception("unrecognized model type, see help documentation")

    return gen_sklearn_model(model_config, xgb_params)

## train

In [6]:
def train_model(
    context: MLClientCtx,
    model_type: str,
    dataset: DataItem,
    label_column: str = "labels",
    sample: int = -1,
    test_size: float = 0.05,
    valid_size: float = 0.75,
    random_state: int = 1,
    model_filename: str = "xgb-model",
    models_dest: str = "",
    plots_dest: str = "",
    score_method: str = "micro",
    file_ext: str = "parquet",
    model_pkg_file: str = "",    
) -> None:
    """train an xgboost model.

    :param context:           the function context
    :param model_type:        the model type to train, 'classifier', 'regressor'...
    :param dataset:           ("data") name of raw data file
    :param label_column:      ground-truth (y) labels
    :param sample:            Selects the first n rows, or select a sample
                              starting from the first. If negative <-1, select
                              a random sample
    :param model_filename:    model file filename,
                              points to a directory
    :param test_size:         (0.05) test set size
    :param valid_size:          (0.75) Once the test set has been removed the
                              training set gets this proportion.
    :param random_state:      (1) sklearn rng seed
    :param models_dest:       models subfolder on artifact path
    :param plots_dest:        plot subfolder on artifact path
    :param score_method:      for multiclass classification
    
    :param file_ext:          format for test_set_key hold out data
    """
    # deprecate:
    models_dest = models_dest or "models"
    plots_dest = plots_dest or f"plots/{context.name}"
    
    # get a sample from the raw data
    raw, labels, header = get_sample(dataset, sample, label_column)
    
    # split the sample into train validate, test and calibration sets:
    (xtrain,ytrain), (xvalid,yvalid), \
    (xtest,ytest), (xcal, ycal) = get_splits(
        raw, labels, 4,
        test_size, 
        valid_size, 
        ["labels"],
        random_state)
        
    # get model config
    model_config = gen_xgb_model(model_type, context.parameters.items())
    model_short_name = model_config["META"]["class"].split('.')[-1]
   
    # create model instance
    XGBBoostClass = create_class(model_config["META"]["class"])
    model = XGBBoostClass(**model_config["CLASS"])

    # update the model config with training data and callbacks
    model_config["FIT"].update({"X": xtrain,"y": ytrain.values})
    
    # run the fit
    model.fit(**model_config["FIT"])

    # evaluate model
    eval_metrics = eval_class_model(xvalid, yvalid, model)
    model_plots = eval_metrics.pop("plots")
    # just do this inside log_model?
    for plot in model_plots:
        context.log_artifact(plot, local_path=f'plots_dest/{plot.key}.html')
        
    # serialize the model
    model_bin = dumps(model.get_booster())
    context.log_model(model_short_name, body=model_bin, 
                      model_dir="models", 
                      model_file='model.pkl',
                      metrics=eval_metrics)
    # cant see them from log_model so try this:
    context.log_results(eval_metrics)

In [7]:
# nuclio: end-code

### mlconfig

In [8]:
from mlrun import mlconf
import os
mlconf.dbpath = mlconf.dbpath or 'http://mlrun-api:8080'
mlconf.artifact_path = mlconf.artifact_path or f'{os.environ["HOME"]}/artifacts'

### save

In [9]:
from mlrun import code_to_function 
# create job function object from notebook code
fn = code_to_function("xgb_trainer")

# add metadata (for templates and reuse)
fn.spec.default_handler = "train_model"
fn.spec.description = "train any classifier using scikit-learn's API"
fn.metadata.categories = ["models", "classifier"]
fn.metadata.labels = {"author": "yjb"}

fn.save()
fn.export("function.yaml")

[mlrun] 2020-05-03 02:30:50,939 warning!, server (0.4.6) and client (0.4.7) ver dont match
[mlrun] 2020-05-03 02:30:50,972 function spec saved to path: function.yaml


### test function

In [10]:
if "V3IO_HOME" in list(os.environ):
    # mlrun on the iguazio platform
    from mlrun import mount_v3io
    fn.apply(mount_v3io())
else:
    # mlrun is setup using the instructions at 
    # https://github.com/mlrun/mlrun/blob/master/hack/local/README.md
    from mlrun.platforms import mount_pvc
    fn.apply(mount_pvc("nfsvol", "nfsvol", "/home/joyan/data"))

In [11]:
gpus = False

task_params = {
    "name" : "tasks xgb cpu trainer",
    "params" : {
        "model_type"         : "classifier",
        "num_class"          : 5,
        "CLASS_tree_method"  : "gpu_hist" if gpus else "hist",
        "CLASS_objective"    : "multi:softmax",
        'CLASS_n_estimators' : 100,
        "CLASS_max_depth"    : 6,
        "CLASS_booster"      : "gbtree",  
        "CLASS_random_state" : 1,
        "CLASS_base_score" : 0.5,
        "CLASS_colsample_bylevel" : 1,
        "CLASS_colsample_bynode" : 1,
        "CLASS_colsample_bytree" : 1,
        "CLASS_scale_pos_weight" : 1,
        "CLASS_min_child_weight" : 1,
        "sample"             : -1,
        "label_column"       : "labels",
        "test_size"          : 0.10,
        "valid_size"         : 0.75,
        "score_method"       : "weighted",
        "models_dest"        : "xgb_trainer/models",
        "plots_dest"         : "xgb_trainer/plots",
    }}

### run locally

In [ ]:
from mlrun import run_local, NewTask

run = run_local(
    NewTask(**task_params),
    handler=train_model,
    inputs={"dataset"  : os.path.join(mlconf.artifact_path, "classifier-data.parquet")})

### run remotely

In [13]:
# run = fn.run(
#     NewTask(**task_params),
#     inputs={"dataset"  : os.path.join(mlconf.artifact_path, "breast_cancer.parquet")})

## tests

In [14]:
# WIP


# def test_gen_xgb_model():
#     import xgboost
#     c, j = _gen_xgb_model("rf_classifier", {})
#     assert isinstance(c, xgboost.XGBRFClassifier)
# test_gen_xgb_model()

# breast_cancer = get_dataitem(mlconf.artifact_path+"/breast_cancer.parquet")
# classifier-data = get_dataitem(mlconf.artifact_path+"/classifier-data.csv")
# iris = get_dataitem(mlconf.artifact_path+"/iris.parquet")

# def test_get_sample():
#     from mlrun import mlconf
#     r, l, h = _get_sample(breast_cancer, -1, "labels")
#     assert r.shape[0]==l.shape[0]
# test_get_sample()

# def test_get_splits():
#     from mlrun import mlconf
#     r, l, h = _get_sample(classifier-data, -1, "labels")
#     (xtr, ytr), (xva, yva), (xte, yte), (xcal, ycal) = _get_splits(r, l, 4)

#     assert xtr.shape[0]+xva.shape[0]+xte.shape[0]+xcal.shape[0] == r.shape[0]
# test_get_splits()

# #def test_save_test_set():
# r, l, h = _get_sample(iris, -1, "labels")
# A = _get_splits(r,l,3)
# from mlrun import get_or_create_ctx
# _save_test_set(get_or_create_ctx("test"), A[2][1], A[2][1], h, debug=True)
# import pandas as pd
#     # pd.read_parquet()
#     # assert
# #test_save_test_set()